In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pathlib
import os
import io
import cv2
import pickle

W1208 14:36:43.934505  1600 deprecation.py:323] From c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\compat\v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
model_path = "C:/Users/asus/Desktop/Major/car_detection_model"


In [4]:
%%capture
session = tf.Session(graph=tf.Graph())
# loading the model into our session created above
tf.saved_model.loader.load(session, ['serve'], model_path)

In [5]:
# cam = cv2.VideoCapture("./11.png")
# slots = []
# n = 100;
# while True:
#     _, im0 = cam.read()
#     showCrosshair = False
#     fromCenter = False
#     r = cv2.selectROI("Image1", im0, fromCenter, showCrosshair)
#     n = n-1
#     slots.append(r)
#     for slot in slots:
#         im0 = cv2.rectangle(im0, (int(slot[0]),int(slot[1])), 
#                                     (int(slot[0]+slot[2]),int(slot[1]+slot[3])), (255, 0, 0)) 
#     cv2.imshow("Image", im0)
    
#     if cv2.waitKey(0) & 0xFF == ord('q'):
#         break
        
# cam.release()
# cv2.destroyAllWindows()

In [6]:
def draw_boxes(height, width, box, img):
    ymin = int(max(1, (box[0] * height)))
    xmin = int(max(1, (box[1] * width)))
    ymax = int(min(height, (box[2] * height)))
    xmax = int(min(width, (box[3] * width)))
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1)

In [7]:
def predict(img_name,img):
    result = session.run(['detection_boxes:0','detection_scores:0'], feed_dict={
            'encoded_image_string_tensor:0': [img_name]})     
    boxes = result[0][0]
    scores = result[1][0]
    
    imH, imW, _ = img.shape
    b = []
    for i in range(len(scores)):
        if scores[i] > 0.5:
            draw_boxes(imH, imW, boxes[i], img)
            b.append(boxes[i])
            
#     new_img = cv2.resize(img, (1080, 1920))
    # show the image on the screen
#     cv2.imshow("i", img)
#     # as soon as any key is pressed, skip to the next lin
#     cv2.waitKey(0)
    if(len(b)==0):
        return False
    else:
        return True
    
    
    

In [8]:
pickle_in = open("slots_demo.pickle","rb")
slots = pickle.load(pickle_in)

In [10]:
cam = cv2.VideoCapture("./demo.mp4")
i=0
while True:
    ret, im0 = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    i = i + 1
    if i%60!=0:
        continue
    print(i)
    for ids in slots.keys():
        slot = slots[ids]
        final = im0
        final = final*0
        x,y,w,h = slot
        slot_img = im0[y:y+h,x:x+w]
        final[y:y+h,x:x+w] = im0[y:y+h,x:x+w]
#         cv2.imshow("slot"+str(i),final)
        
        img_str = cv2.imencode('.jpg', final)[1].tostring()
        
        found = predict(img_str,final)
        
        if found:
            im0 = cv2.rectangle(im0, (int(x),int(y)), 
                                    (int(x+w),int(y+h)), (255, 0, 0))
        else:
            im0 = cv2.rectangle(im0, (int(x),int(y)), 
                                    (int(x+w),int(y+h)), (0, 255, 0))
    
#     new_img = cv2.resize(im0, (416, 416))
    cv2.imshow("final", im0)
    
    
    
        
cam.release()
cv2.destroyAllWindows()

60
120
180
240
300
360
420
480
540
600
660
720
780
840
900
960
1020
